In [1]:
import zipfile
import requests
import pandas as pd
import numpy as np
from datetime import datetime 
import warnings
warnings.filterwarnings("ignore")        
import datetime
import BS
import Config
from pymongo import MongoClient
import time
from os import listdir
from os.path import isfile, join

client = MongoClient(Config.DB_Hostname, Config.DB_Port)

def Upload_Bhavcopy():
    path="/Users/praneetshaw/Desktop/Strategy/Strategy_FIles/BhavCopy/Bhav/"
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    onlyfiles = [s for s in onlyfiles if ".csv" in s]
    dates=[ datetime.datetime.strptime(o[2:11],"%d%b%Y") for o in onlyfiles]
    onlyfiles= [o for d,o in sorted(zip(dates,onlyfiles))]
    flag=0
    for h in onlyfiles:
                    

        try:
            with zipfile.ZipFile(f'{path+h}', 'r') as zip_ref:
                zip_ref.extractall('BhavCopy')

            try: 
                try:
                    bhavcopy = pd.read_csv(f"BhavCopy/{h[:-4]}")
                except Exception as e:
                    bhavcopy = pd.read_csv(f"BhavCopy/{h[:-4]}/{h[:-4]}")

                bhavcopy["Expiry_Today"]=np.where(bhavcopy.EXPIRY_DT.str.upper()==bhavcopy.TIMESTAMP,1,0)
                bhavcopy["CLOSE"]=np.where((bhavcopy.OPTION_TYP=="CE")&(bhavcopy.Expiry_Today==1),np.maximum(bhavcopy.SETTLE_PR-bhavcopy.STRIKE_PR,0),bhavcopy.CLOSE)
                bhavcopy["CLOSE"]=np.where((bhavcopy.OPTION_TYP=="PE")&(bhavcopy.Expiry_Today==1),np.maximum(bhavcopy.STRIKE_PR-bhavcopy.SETTLE_PR,0),bhavcopy.CLOSE)
                bhavcopy["CLOSE"]=np.where((bhavcopy.OPTION_TYP=="XX")&(bhavcopy.Expiry_Today==1),bhavcopy.SETTLE_PR,bhavcopy.CLOSE)
                bhavcopy=bhavcopy[[ 'INSTRUMENT','SYMBOL', 'EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN','HIGH', 'LOW', 'CLOSE', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH','OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP',]]

                print(client.Strategy.Bhavcopy_Subset.delete_many({"TIMESTAMP":h[2:11][:2]+"-"+h[2:11][2:5]+"-"+h[2:11][-4:]}).deleted_count,f" documents deleted for {h[2:11]}")
                print(len(client.Strategy.Bhavcopy_Subset.insert_many(bhavcopy.to_dict('records')).inserted_ids),f" documents entered for {h[2:11]}\n")

            except Exception as e:
                print(e,f"Could not read {h[:-4]}")

        except Exception as e:
            print(e,f". Bhav Copy is not available for {h}")


def Download_Bhavcopy(date_today):
    
    date=date_today[:2]+"-"+date_today[2:5]+"-"+date_today[-4:]
    print(date)
    return pd.DataFrame(pd.DataFrame(client.Strategy.Bhavcopy_Subset.find({"TIMESTAMP":date})))

def Process_Bhavcopy(bhavcopy,date_today,hdays=[]):
    
    start=time.time()
    bhavcopy.EXPIRY_DT=pd.to_datetime(bhavcopy.EXPIRY_DT)
    total_summary=pd.DataFrame(columns=['Underlying', 'Date', 'IV_Current', 'IV_Next'])
    date_today_formatted=datetime.datetime.strftime(datetime.datetime.strptime(date_today, '%d%b%Y'),"%Y-%m-%d")
    
    for Underlying in ["NIFTY", "BANKNIFTY"]:
        try:
            bhavcopy_sub=bhavcopy[(bhavcopy.SYMBOL==Underlying)]
            summary={}
            summary["Underlying"]=Underlying
            summary["Date"]=date_today_formatted
            bhavcopy_sub_FUT=bhavcopy_sub[bhavcopy_sub.OPTION_TYP=="XX"]
            bhavcopy_sub_OPT=bhavcopy_sub[bhavcopy_sub.OPTION_TYP!="XX"]
            bhavcopy_sub_FUT.sort_values(by='EXPIRY_DT',inplace=True)
            bhavcopy_sub_OPT.sort_values(by='EXPIRY_DT',inplace=True)
            Future_open=float(bhavcopy_sub_FUT.OPEN[0:1])
            bhavcopy_sub_OPT=bhavcopy_sub_OPT[bhavcopy_sub_OPT.EXPIRY_DT.isin(bhavcopy_sub_OPT.EXPIRY_DT.sort_values().unique()[0:2])]
            bhavcopy_sub_OPT["Time_To_Expiry"]=bhavcopy_sub_OPT.EXPIRY_DT.apply(lambda x: np.busday_count(datetime.datetime.strptime(date_today, '%d%b%Y').date(),x.date(),holidays=hdays))
            bhavcopy_sub_OPT["Strike_FUT_Diff_Open"]=abs(bhavcopy_sub_OPT.STRIKE_PR-Future_open)
            temp=bhavcopy_sub_OPT.groupby('EXPIRY_DT').Strike_FUT_Diff_Open.min()
            bhavcopy_open_options=bhavcopy_sub_OPT[bhavcopy_sub_OPT.Strike_FUT_Diff_Open.isin(temp)]
            bhavcopy_open_options["IVOL"]=bhavcopy_open_options[['OPEN','STRIKE_PR','Time_To_Expiry','OPTION_TYP']].apply(lambda x : BS.implied_volatility(x.OPEN,Future_open,x.STRIKE_PR,(x.Time_To_Expiry+1)/252,Config.interest_rate,x.OPTION_TYP), axis=1)
            bhavcopy_open_options.IVOL.fillna(bhavcopy_open_options.IVOL.mean(),inplace=True)
            bhavcopy_open_options.IVOL.fillna(0,inplace=True)
            summary["IV_Current"]=bhavcopy_open_options[bhavcopy_open_options.EXPIRY_DT==bhavcopy_open_options.EXPIRY_DT.unique()[0]].IVOL.mean()
            summary["IV_Next"]=bhavcopy_open_options[bhavcopy_open_options.EXPIRY_DT==bhavcopy_open_options.EXPIRY_DT.unique()[1]].IVOL.mean()
            total_summary=pd.concat([total_summary,pd.DataFrame([summary])])
        except Exception as e:
            print(Underlying,e)
            
    print(client.Strategy.Daily_Bhavcopy.delete_many({"Date":date_today_formatted}).deleted_count,f" documents deleted for {date_today_formatted}")
    print(len(client.Strategy.Daily_Bhavcopy.insert_many(total_summary.to_dict('records')).inserted_ids),f" documents entered for {date_today_formatted}\n")
    print("Processing Time: ",time.time()-start)

In [2]:
Upload_Bhavcopy()

0  documents deleted for 21FEB2024
54826  documents entered for 21FEB2024

0  documents deleted for 22FEB2024
54910  documents entered for 22FEB2024

0  documents deleted for 23FEB2024
55038  documents entered for 23FEB2024

0  documents deleted for 26FEB2024
55134  documents entered for 26FEB2024



In [3]:
start_date="21FEB2024"
end_date="27FEB2024"
date=datetime.datetime.strptime(start_date, '%d%b%Y')
holidays=pd.DataFrame(client.Strategy.Holidays.find())
holidays=[d.date() for d in pd.to_datetime(holidays.date)]

while(date!=datetime.datetime.strptime(end_date, '%d%b%Y')): 
    
    date_today=datetime.datetime.strftime(date,"%d%b%Y").upper()

    start=time.time()
    bhavcopy=Download_Bhavcopy(date_today)
    print("Downloading_Time: ",time.time()-start)
    
    if(len(bhavcopy)>0):
        Process_Bhavcopy(bhavcopy, date_today, hdays=holidays)
    else:
        print(f"Could not process for {date_today}")
    date += datetime.timedelta(days=1)

21-FEB-2024
Downloading_Time:  0.7310910224914551
0  documents deleted for 2024-02-21
2  documents entered for 2024-02-21

Processing Time:  0.4410669803619385
22-FEB-2024
Downloading_Time:  0.5799009799957275
0  documents deleted for 2024-02-22
2  documents entered for 2024-02-22

Processing Time:  0.4377620220184326
23-FEB-2024
Downloading_Time:  0.5773019790649414
0  documents deleted for 2024-02-23
2  documents entered for 2024-02-23

Processing Time:  0.423555850982666
24-FEB-2024
Downloading_Time:  0.21693801879882812
Could not process for 24FEB2024
25-FEB-2024
Downloading_Time:  0.1711282730102539
Could not process for 25FEB2024
26-FEB-2024
Downloading_Time:  0.5496101379394531
0  documents deleted for 2024-02-26
2  documents entered for 2024-02-26

Processing Time:  0.42804479598999023


In [17]:
df = pd.DataFrame(client.Strategy.Bhavcopy_Subset.find()).drop(columns = ['_id'])
df = df[df.SYMBOL.isin(["NIFTY", "BANKNIFTY"])]


In [18]:
print(len(client.Strategy.Bhavcopy.insert_many(df.to_dict('records')).inserted_ids))


58245
